In [1]:
from PIL import Image
import numpy as np
#引入torch，torchvision库
import torch
from torchvision import datasets,transforms
#datasets是加载图片数据的方法，transforms是图像预处理的方法
from torch import nn,optim
#nn即neural network，是专门为神经网络设计的模块化接口；optim是优化器
import torch.nn.functional as F

D:\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.Compose([  transforms.ToTensor(),  transforms.Normalize((0.1307,),(0.3081,))])
#ToTensor（）是将数据转化为Tensor对象，Normalize（）是对数据进行归一化

In [3]:
trainset = datasets.MNIST('data', train=True, download=False, transform=transform)
testset = datasets.MNIST('data', train=False, download=False, transform=transform)
#使用datasets.MNIST（）来分别下载训练数据集和测试数据集

In [11]:
#构建LeNet模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        #卷积层c1，c3
        self.c1 = nn.Conv2d(1,6,(5,5))
        self.c3 = nn.Conv2d(6,16,(5,5))
        #全连接层fc1，2，3
        self.fc1 = nn.Linear(16*4*4,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
    #池化，池化核2x2
        x = F.max_pool2d(F.relu(self.c1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.c3(x)),(2,2))
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *=s
        return num_features
    

In [12]:
#初始化LeNet模型
CUDA = 0
if CUDA:
    lenet = LeNet().cuda()
else:
    lenet = LeNet()
    
#损失函数为交叉熵函数,优化器为随机梯度下降
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(),lr=0.001,momentum=0.9)

In [5]:
#batch_size表示一次加载的数据量，shuffle = True表示遍历不同批次的数据时打乱顺序，num_workers=2表示使用两个子进程加载数据
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle = True, num_workers=2)
for i,data in enumerate(trainloader,0):
    inputs,labels = data
    print(labels)
    if i == 1:
        break

tensor([9, 4, 2, 4])
tensor([1, 9, 2, 7])


In [7]:
#训练
def train(model,criterion,optimizer,epochs=1):
    for epoch in range(epochs):
        running_loss = 0
        for i,data in enumerate(trainloader,0):
            inputs,labels = data
            if CUDA:
                inputs,labels = inputs.cuda(),labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            if i%1000==999:
                print('[Epoch:%d,Batch:%5d] Loss: %.3f' % (epoch+1, i+1, running_loss/1000))
                running_loss = 0.0

train(lenet,criterion,optimizer,epochs =1)

[Epoch:1,Batch: 1000] Loss: 1.295
[Epoch:1,Batch: 2000] Loss: 0.305
[Epoch:1,Batch: 3000] Loss: 0.221
[Epoch:1,Batch: 4000] Loss: 0.161
[Epoch:1,Batch: 5000] Loss: 0.157
[Epoch:1,Batch: 6000] Loss: 0.146
[Epoch:1,Batch: 7000] Loss: 0.113
[Epoch:1,Batch: 8000] Loss: 0.115
[Epoch:1,Batch: 9000] Loss: 0.110
[Epoch:1,Batch:10000] Loss: 0.112
[Epoch:1,Batch:11000] Loss: 0.096
[Epoch:1,Batch:12000] Loss: 0.095
[Epoch:1,Batch:13000] Loss: 0.090
[Epoch:1,Batch:14000] Loss: 0.083
[Epoch:1,Batch:15000] Loss: 0.079


In [8]:
#保存训练好的模型，命名model.pkl
torch.save(lenet.state_dict(), 'model.pkl')

In [9]:
#加载测试集
testset = datasets.MNIST('data', train=False, download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=False, num_workers = 2)
#测试训练完的模型
def test(testloader,model):
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        if CUDA:
            images = images.cuda()
             labels = labels.cuda()
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total +=labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy:%d %%' % (100*correct/total))

lenet.load_state_dict(torch.load('model.pkl'))
test(testloader,lenet)


Accuracy:97 %


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [17]:
'''i = 0
for data in testloader:
    images, labels = data
    print(lenet(images))
    print(labels)
    images = torch.squeeze(images,0)
    to_pil_img = transforms.ToPILImage()#tensor 重新转化成图片格式
    images = to_pil_img(images)
    images.show()
    i+=1
    if i>=1:
        break

tensor([[ -2.8601,  -2.3236,   5.1342,   3.4096,  -1.8630,  -4.5394, -10.1022,
          17.1239,  -3.1820,   3.1620]], grad_fn=<AddmmBackward>)
tensor([7])


In [47]:
import cv2
#自己手写数字来进行测试
#读取训练好的模型
lenet = LeNet()
lenet.load_state_dict(torch.load('model.pkl'))
#输入图片地址来读-取图片。返回torch.tensor
def image_loader(image_name):
    image = cv2.imread(image_name)#读取图片
    #转换图片为灰度图片
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #高斯滤波
    gauss_img = cv2.GaussianBlur(gray_image,(5,5),0,0,cv2.BORDER_DEFAULT)
    #修改图片大小为28*28像素
    image = cv2.resize(gauss_img,(28,28))

    for h in range(28):
        for w in range(28):
            image[h][w] = 255 - image[h][w]
    to_pil_img = transforms.ToPILImage()#tensor 重新转化成图片格式
    image1 = to_pil_img(image)
    image1.show()
    #归一化像素数据
    image = transform(image)
    return torch.unsqueeze(image.to(torch.float),0)


image = image_loader('test_9.jpg')
print(image.size())
print(lenet(image))
#tensor 重新转化成图片格式
image = torch.squeeze(image,0)
to_pil_img = transforms.ToPILImage()#tensor 重新转化成图片格式
image = to_pil_img(image)
#image.show()

torch.Size([1, 1, 28, 28])
tensor([[-0.9180,  1.2726,  2.1164,  0.8098, -0.8048, -1.5139, -2.6360,  3.2061,
         -0.5543,  0.0556]], grad_fn=<AddmmBackward>)


In [5]:
import torch
CUDA = torch.cuda.is_available()
print(torch.cuda.device_count())

1


In [12]:
import torch
a = torch.tensor([1,2,3,4]).unsqueeze(0)
b = torch.tensor([1,2,3,4]).unsqueeze(0)
c = torch.cat((a,b),0)
print(a.size())
print(c)

torch.Size([1, 4])
tensor([[1, 2, 3, 4],
        [1, 2, 3, 4]])


In [19]:
image = cv2.imread('test_0.jpg')#读取图片
print(image.type())

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [14]:
import random
epochs = 10000
sum1 = 0
sum2 = 0
sum3 = 0

for epoch in range(epochs):
    i = random.randrange(1,4)
    if i == 1:
        sum1 += 1
    elif i == 2:
        sum2 += 1
    else:
        sum3 += 1
print('1，2，3三个数字出现的频率分别为%.4f%%,%.4f%%,%.4f%%。'%(100*sum1/epochs,100*sum2/epochs,100*sum3/epochs))

1，2，3三个数字出现的频率分别为0.3252%,0.3369%,0.3379%。


In [ ]:
def get_scales(image_root,scale):
    #每次图片缩小的比例定为缩小后的图片面积为原来的一半
    scale = scale
    with Image.open(r'F:\PyTorch\人脸检测识别\数据集\CeleA\Img\img_align_celeba\000001.jpg') as img:
    copy = img.copy()
    print(img.size)

def Pnet_Process(image,scales):
    

In [32]:
from PIL import Image
from torchvision import transforms
squeeze = transforms.ToTensor()

with Image.open(r'F:\PyTorch\人脸检测识别\数据集\CeleA\Img\img_align_celeba\000001.jpg') as img:
    copy = img.copy()
    print(img.size)

(178, 218)
